# Exploratory Data Analysis
Understanding the dataset to explore how the data is present in the database and if there is needed of creating some aggregated tables that can help with:

  - Vendor selection for Profitability
  - Product Pricing Optimization

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
import sqlite3
from scipy.stats import ttest_ind
import scipy.stats as stats
warnings.filterwarnings('ignore')

### Loading Dataset

In [56]:
# Creating daatabase connection
conn=sqlite3.connect('inventory.db')

In [3]:
# show the table present in database file (inventory.db)
tables=pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn)
tables

,name
0,purchase_price
1,sales
2,vendor_invoice
3,begin_inventory
4,end_inventory
5,purchases


In [4]:
# total record and first five row  in each table 
for table in tables['name']:
    print("-"*50,f'{table}','-'*50)
    print("count of records:",pd.read_sql_query(f"SELECT COUNT(*) AS 'count' FROM {table}",conn)['count'].values[0])
    display(pd.read_sql(f"SELECT * FROM {table} LIMIT 5 ",conn))

-------------------------------------------------- purchase_price --------------------------------------------------
count of records: 12261


,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


-------------------------------------------------- sales --------------------------------------------------
count of records: 12825363


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


-------------------------------------------------- vendor_invoice --------------------------------------------------
count of records: 5543


...

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
count of records: 224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- purchases --------------------------------------------------
count of records: 2372474


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


In [5]:
# cheack the file record and sql fetch data record is same
for file in os.listdir('inventory dataset'):
    if '.csv' in file:
        df=pd.read_csv('inventory dataset/'+file)
        print(df.shape)
        print("-"*50,f'{file}','-'*50)


(206529, 9)
-------------------------------------------------- begin_inventory.csv --------------------------------------------------
(224489, 9)
-------------------------------------------------- end_inventory.csv --------------------------------------------------
(2372474, 16)
-------------------------------------------------- purchases.csv --------------------------------------------------
(12261, 9)
-------------------------------------------------- purchase_prices.csv --------------------------------------------------
(12825363, 14)
-------------------------------------------------- sales.csv --------------------------------------------------
(5543, 10)
-------------------------------------------------- vendor_invoice.csv --------------------------------------------------


In [6]:
# see the purchases of a vendor 
purchases=pd.read_sql_query("SELECT * from purchases where VendorNumber=4466" ,conn)
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [7]:
# see the purchase_price of a vendor 

purchase_price=pd.read_sql_query("SELECT * FROM purchase_price WHERE VendorNumber=4466",conn)
purchase_price

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [8]:
 #see the invoice of a vendor 

vendor_invoice=pd.read_sql_query("SELECT * FROM vendor_invoice  WHERE VendorNumber=4466",conn)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [9]:
# sum of total purchase and total quantity
purchases.groupby(["Brand","PurchasePrice"])[["Dollars","Quantity"]].sum()

,,Dollars,Quantity
Brand,PurchasePrice,,
3140,11.19,51921.60,4640
5215,9.41,46325.43,4923
5255,9.35,58110.25,6215


In [10]:
purchase_price

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [11]:
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [12]:
sale=pd.read_sql_query("SELECT * FROM sales WHERE VendorNO=4466",conn)
sale

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9448,9_BLACKPOOL_5215,9,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-12-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9449,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-02,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9450,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9451,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [13]:
sale.head(1)

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [14]:
# sum of total purchase and total quantity by vendor
purchases.groupby(["Brand","PurchasePrice"])[["Dollars","Quantity"]].sum()

,,Dollars,Quantity
Brand,PurchasePrice,,
3140,11.19,51921.60,4640
5215,9.41,46325.43,4923
5255,9.35,58110.25,6215


In [15]:
# sum of total sale  by vendor
sale.groupby('Brand')[['SalesQuantity','SalesDollars','SalesPrice']].sum()

,SalesQuantity,SalesDollars,SalesPrice
Brand,,,
3140,3890,50531.10,30071.85
5215,4651,60416.49,41542.02
5255,6096,79187.04,51180.60


- The purchase table contains actual purchase data,including the date of purchase,product(brand) purchase by vendor ,the amount paid(in dollars),and the quantity purchased
- The purchase_price column is derived from the purchase_price table ,which provide the product_wise actual and purchase prices. The combination of vendor and brand is unique in this table
- The vendor_invoice table aggregates data from the purchases tables ,summarizing quantity and dollar amounts,along with an additional column for freight,This table manitains the uniqueness based on vendor and PO number
- The sales  table captures actual transection,detailing the brand purchased by vendor ,the quantity sale by vendor,selling price and revenue earned.

##### As the data that we need for analysis is distributed in different table , we need to create a summary table containing:
- purchase transection made by vendors
- sales transection data
- freight costs for each vendor
- actual product prices from vendors

In [16]:
# find the total freight cost of each vendor
freight_summary=pd.read_sql_query("""SELECT VendorNumber,SUM(Freight) AS "total_freight" 
                                   FROM vendor_invoice
                                  GROUP BY VendorNumber""",conn)
freight_summary

,VendorNumber,total_freight
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [17]:
#  find the total Quantity Purchases,total purchase amount,
purchase_summary=pd.read_sql_query(""" SELECT 
                                  t1.VendorNumber,t1.VendorName,t1.Brand,
                                    t1.PurchasePrice,
                                    t2.Price,t2.Volume,SUM(t1.Quantity) AS 'total_Quantity',SUM(t1.Dollars) AS "total_purchase_amount"
                                   FROM purchases t1

                                   JOIN purchase_price t2
                                   ON t1.Brand=t2.Brand
                                   WHERE t1.PurchasePrice>0
                                   GROUP BY t1.VendorNumber,t1.VendorName,t1.brand
                                   ORDER BY total_purchase_amount
                                   """,conn)
purchase_summary

,VendorNumber,VendorName,Brand,PurchasePrice,Price,Volume,total_Quantity,total_purchase_amount
0,7245,PROXIMO SPIRITS INC.,3065,0.71,0.99,50,1,0.71
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,1.99,200,1,1.47
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,0.99,50,2,1.48
3,8004,SAZERAC CO INC,5683,0.39,0.49,50,6,2.34
4,9815,WINE GROUP INC,8527,1.32,4.99,750,2,2.64
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,29.99,1750,138109,3023206.01
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,22.99,1750,201682,3261197.94
10689,17035,PERNOD RICARD USA,8068,18.24,24.99,1750,187407,3418303.68
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,28.99,1750,164038,3804041.22


In [18]:
sale_summary = pd.read_sql_query("""
SELECT 
    VendorNo,
    Brand,
    SUM(SalesQuantity) AS total_Quantity,
    SUM(SalesDollars) AS total_sale_dollar,
    SUM(SalesPrice) AS total_sale_price,
    SUM(ExciseTax) AS total_excise_tax
FROM sales
GROUP BY 
    VendorNo,
    Brand
ORDER BY total_sale_dollar ;
""", conn)

sale_summary


,VendorNo,Brand,total_Quantity,total_sale_dollar,total_sale_price,total_excise_tax
0,8004,5287,2,9.800000e-01,0.98,0.10
1,9206,2773,1,9.900000e-01,0.99,0.05
2,3252,3933,2,1.980000e+00,0.99,0.10
3,3924,9123,2,1.980000e+00,0.99,0.10
4,10050,3623,2,1.980000e+00,1.98,0.10
...,...,...,...,...,...,...
11267,3960,3545,135838,4.223108e+06,545778.28,249587.83
11268,3960,4261,200412,4.475973e+06,420050.01,368242.80
11269,17035,8068,187140,4.538121e+06,461140.15,343854.07
11270,4425,3405,160247,4.819073e+06,561512.37,294438.66


##### we fetch the summary_sale from database using join tables((purchase,purchase_price),sale,vendor_invoice). you can also join all three(freight_summary,purchase_summary,sale_summary) table using pandas but join will be slow in pandas  


In [8]:
import time

start = time.time()
sale_summary = pd.read_sql_query("""
WITH 
freight_summary AS (SELECT 
                               VendorNumber,
                               SUM(Quantity) AS total_quantity_invoice,
                               SUM(Dollars) AS total_dollar_invoice,
                               SUM(Freight) AS total_freight_cost
                               FROM vendor_invoice
                               GROUP BY VendorNumber),
purchase_summary AS (
    SELECT
        t1.VendorNumber,
        t1.VendorName,
        t1.Brand,
        t2.Description,
        t2.Volume,
        t2.Price AS actual_price,
        t2.PurchasePrice,	
        SUM(t1.Quantity) AS total_quantity_purchase,
        SUM(t1.Dollars) AS total_dollar_purchase
    FROM purchases t1
    JOIN purchase_price t2
        ON t1.VendorNumber = t2.VendorNumber 
        AND t1.Brand = t2.Brand
    WHERE t1.PurchasePrice > 0
    GROUP BY t1.VendorNumber, t1.Brand, t1.VendorName, t2.Volume, t2.Price, t2.PurchasePrice,t2.Description
),
sale_summary AS (
    SELECT 
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS total_Quantity_sale,
        SUM(SalesDollars) AS total_sale_dollar,
        SUM(SalesPrice) AS total_sale_price,
        SUM(ExciseTax) AS total_excise_tax
    FROM sales
    GROUP BY VendorNo, Brand
)
SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.Description,
    p.Volume,
    p.actual_price,
    p.PurchasePrice,
    p.total_quantity_purchase,
    p.total_dollar_purchase,
    s.total_Quantity_sale,
    s.total_sale_price,
    s.total_sale_dollar,
    s.total_excise_tax,
    f.total_quantity_invoice,
    f.total_dollar_invoice,
    f.total_freight_cost
FROM purchase_summary p
LEFT JOIN sale_summary s
    ON p.VendorNumber = s.VendorNo AND p.Brand = s.Brand
LEFT JOIN freight_summary f
    ON p.VendorNumber = f.VendorNumber;
""", conn)
end = time.time()

print("Execution time:", end - start)



Execution time: 146.59491443634033


In [55]:
sale_summary

,VendorNumber,VendorName,Brand,Description,Volume,actual_price,PurchasePrice,total_quantity_purchase,total_dollar_purchase,total_Quantity_sale,total_sale_price,total_sale_dollar,total_excise_tax,total_quantity_invoice,total_dollar_invoice,total_freight_cost,gross_profit,margin,turnOver,sale_to_purchase_ratio
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",90085,Ch Lilian 09 Ladouys St Este,750.0,36.99,23.86,8,190.88,18.0,295.92,665.82,2.00,328,5630.88,27.08,474.94,0.713316,0.444444,3.488160
1,2,"IRA GOLDMAN AND WILLIAMS, LLP",90609,Flavor Essence Variety 5 Pak,162.5,24.99,17.00,320,5440.00,24.0,449.82,599.76,0.52,328,5630.88,27.08,-4840.24,-8.070295,13.333333,0.110250
2,54,AAPER ALCOHOL & CHEMICAL CO,990,Ethyl Alcohol 200 Proof,3750.0,134.49,105.07,1,105.07,0.0,0.00,0.00,0.00,1,105.07,0.48,-105.07,-inf,inf,0.000000
3,60,ADAMBA IMPORTS INTL INC,771,Bak's Krupnik Honey Liqueur,750.0,14.99,11.44,39,446.16,47.0,494.67,704.53,37.01,4732,76770.25,367.52,258.37,0.366727,0.829787,1.579097
4,60,ADAMBA IMPORTS INTL INC,3401,Vesica Vodka,1750.0,14.99,11.10,6,66.60,0.0,0.00,0.00,0.00,4732,76770.25,367.52,-66.60,-inf,inf,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,172662,SWEETWATER FARM,4215,Kingfish Silver Rum,750.0,25.99,19.40,224,4345.60,129.0,2702.96,3352.71,101.75,1629,34708.03,178.34,-992.89,-0.296146,1.736434,0.771518
10644,173357,TAMWORTH DISTILLING,2804,Camp Robber Whiskey,750.0,44.99,32.14,210,6749.40,140.0,3194.29,6298.60,110.33,1990,41036.44,202.50,-450.80,-0.071571,1.500000,0.933209
10645,173357,TAMWORTH DISTILLING,3666,Art in the Age Chicory Root,375.0,24.99,18.79,520,9770.80,360.0,4873.05,8996.40,141.19,1990,41036.44,202.50,-774.40,-0.086079,1.444444,0.920743
10646,173357,TAMWORTH DISTILLING,3848,Chicory Root Vodka,750.0,30.99,23.30,28,652.40,6.0,92.97,185.94,4.71,1990,41036.44,202.50,-466.46,-2.508659,4.666667,0.285009


##### This querY generete the vendor-wise total_purchses and total sale summary ,which will be use in evalution for :

### Performance Optimization

- The query involve heavy join and aggregation on large dataset lile sale and purchases.
- Storing the pre-aggregated results avoid repeated expensive computations.
- help in analyzing sales,purchases,and pricing adjusment for different vendor.
- Future Benefits of storing this data for faster Dashboording & Reporting.
- Instead of running expensive queries each time,dashboard can fetch data quickly from sale_summary.

In [25]:
# create a main summarise table-->(purchase_summary,Sale_summary,invoice_summary)-->agg_summary
agg_purchase_sale_summary=purchase_summary.merge(sale_summary, left_on=['VendorNumber','Brand'],right_on=['VendorNo',"Brand"],how='left').merge(freight_summary,on='VendorNumber')
agg_purchase_sale_summary

,VendorNumber,VendorName,Brand,PurchasePrice,Price,Volume,total_Quantity_x,total_purchase_amount,VendorNo,total_Quantity_y,total_sale_dollar,total_sale_price,total_excise_tax,total_freight
0,7245,PROXIMO SPIRITS INC.,3065,0.71,0.99,50,1,0.71,7245.0,86.0,8.514000e+01,33.66,4.46,38994.78
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,1.99,200,1,1.47,3960.0,72.0,1.432800e+02,77.61,15.12,257032.07
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,0.99,50,2,1.48,3924.0,2.0,1.980000e+00,0.99,0.10,14069.87
3,8004,SAZERAC CO INC,5683,0.39,0.49,50,6,2.34,8004.0,134.0,6.566000e+01,1.47,7.04,50293.62
4,9815,WINE GROUP INC,8527,1.32,4.99,750,2,2.64,9815.0,5.0,1.595000e+01,10.96,0.55,27100.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,29.99,1750,138109,3023206.01,3960.0,135838.0,4.223108e+06,545778.28,249587.83,257032.07
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,22.99,1750,201682,3261197.94,3960.0,200412.0,4.475973e+06,420050.01,368242.80,257032.07
10689,17035,PERNOD RICARD USA,8068,18.24,24.99,1750,187407,3418303.68,17035.0,187140.0,4.538121e+06,461140.15,343854.07,123780.22
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,28.99,1750,164038,3804041.22,4425.0,160247.0,4.819073e+06,561512.37,294438.66,144929.24


In [28]:
# cheack if there inconsistancy then remove
# cheack data_type
sale_summary.dtypes

VendorNumber                 int64
VendorName                  object
Brand                        int64
Description                 object
Volume                     float64
actual_price               float64
PurchasePrice              float64
total_quantity_purchase      int64
total_dollar_purchase      float64
total_Quantity_sale        float64
total_sale_price           float64
total_sale_dollar          float64
total_excise_tax           float64
total_quantity_invoice       int64
total_dollar_invoice       float64
total_freight_cost         float64
dtype: object

In [12]:
# valume column datatype object-> flaot
sale_summary['Volume']=sale_summary['Volume'].astype(float)

In [29]:
# cheack null values
sale_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
Volume                     0
actual_price               0
PurchasePrice              0
total_quantity_purchase    0
total_dollar_purchase      0
total_Quantity_sale        0
total_sale_price           0
total_sale_dollar          0
total_excise_tax           0
total_quantity_invoice     0
total_dollar_invoice       0
total_freight_cost         0
dtype: int64

In [24]:
# fill null value--> 0   beacouse  vendor purchase the product but not sale 
sale_summary=sale_summary.fillna(0)

In [25]:
# one by one object column verify
sale_summary['VendorName'].unique()     # white spaces show remove it

array(['IRA GOLDMAN AND WILLIAMS, LLP', 'AAPER ALCOHOL & CHEMICAL CO',
       'ADAMBA IMPORTS INTL INC', 'ALTAMAR BRANDS LLC',
       'AMERICAN SPIRITS EXCHANGE', 'APPOLO VINEYARDS LLC',
       'ATLANTIC IMPORTING COMPANY', 'BACARDI USA INC',
       'BANFI PRODUCTS CORP', 'STATE WINE & SPIRITS',
       'SAZERAC NORTH AMERICA INC.', 'BRONCO WINE COMPANY',
       'BROWN-FORMAN CORP', 'BULLY BOY DISTILLERS',
       'BLACK ROCK SPIRITS LLC', 'CALEDONIA SPIRITS INC',
       'CONSTELLATION BRANDS INC', 'CAPSTONE INTERNATIONAL',
       'CASTLE BRANDS CORP.', 'VINEYARD BRANDS INC',
       'VINEYARD BRANDS LLC', 'DIAGEO CHATEAU ESTATE WINES',
       'VRANKEN AMERICA', 'Circa Wines', 'FABRIZIA SPIRITS LLC',
       'ALISA CARR BEVERAGES', 'SOUTHERN WINE & SPIRITS NE',
       'SOUTHERN GLAZERS W&S OF NE', 'DELICATO VINEYARDS INC',
       'BLACK PRINCE DISTILLERY INC', 'DJINN SPIRITS LLC',
       'DUGGANS DISTILLED PRODUCTS', 'DISARONNO INTERNATIONAL LLC',
       'EDRINGTON AMERICAS', 'CENTEUR IMPO

In [23]:
sale_summary['VendorName']=sale_summary['VendorName'].str.strip()

In [27]:
sale_summary['Description'].unique()

array(['Ch Lilian 09 Ladouys St Este', 'Flavor Essence Variety 5 Pak',
       'Ethyl Alcohol 200 Proof', ..., 'Art in the Age Chicory Root',
       'Chicory Root Vodka', 'White Mountain Vodka'], dtype=object)

In [30]:
sale_summary.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'Volume',
       'actual_price', 'PurchasePrice', 'total_quantity_purchase',
       'total_dollar_purchase', 'total_Quantity_sale', 'total_sale_price',
       'total_sale_dollar', 'total_excise_tax', 'total_quantity_invoice',
       'total_dollar_invoice', 'total_freight_cost'],
      dtype='object')

In [69]:
# apply feasure enginnering 
sale_summary['gross_profit']=sale_summary['total_sale_dollar']-sale_summary['total_dollar_purchase']
sale_summary['margin']=np.where(sale_summary['total_sale_dollar']==0,0,sale_summary['gross_profit']/sale_summary['total_sale_dollar'])
sale_summary['sale_to_purchase_ratio']=np.where(sale_summary['total_dollar_purchase']==0,0,sale_summary['total_sale_dollar']/sale_summary['total_dollar_purchase'])
sale_summary['turnOver']=np.where(sale_summary['total_quantity_purchase']==0,0,sale_summary['total_Quantity_sale']/sale_summary['total_quantity_purchase'])

In [70]:
sale_summary

,VendorNumber,VendorName,Brand,Description,Volume,actual_price,PurchasePrice,total_quantity_purchase,total_dollar_purchase,total_Quantity_sale,total_sale_price,total_sale_dollar,total_excise_tax,total_quantity_invoice,total_dollar_invoice,total_freight_cost,gross_profit,margin,sale_to_purchase_ratio,turnOver
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",90085,Ch Lilian 09 Ladouys St Este,750.0,36.99,23.86,8,190.88,18.0,295.92,665.82,2.00,328,5630.88,27.08,474.94,0.713316,3.488160,2.250000
1,2,"IRA GOLDMAN AND WILLIAMS, LLP",90609,Flavor Essence Variety 5 Pak,162.5,24.99,17.00,320,5440.00,24.0,449.82,599.76,0.52,328,5630.88,27.08,-4840.24,-8.070295,0.110250,0.075000
2,54,AAPER ALCOHOL & CHEMICAL CO,990,Ethyl Alcohol 200 Proof,3750.0,134.49,105.07,1,105.07,0.0,0.00,0.00,0.00,1,105.07,0.48,-105.07,0.000000,0.000000,0.000000
3,60,ADAMBA IMPORTS INTL INC,771,Bak's Krupnik Honey Liqueur,750.0,14.99,11.44,39,446.16,47.0,494.67,704.53,37.01,4732,76770.25,367.52,258.37,0.366727,1.579097,1.205128
4,60,ADAMBA IMPORTS INTL INC,3401,Vesica Vodka,1750.0,14.99,11.10,6,66.60,0.0,0.00,0.00,0.00,4732,76770.25,367.52,-66.60,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,172662,SWEETWATER FARM,4215,Kingfish Silver Rum,750.0,25.99,19.40,224,4345.60,129.0,2702.96,3352.71,101.75,1629,34708.03,178.34,-992.89,-0.296146,0.771518,0.575893
10644,173357,TAMWORTH DISTILLING,2804,Camp Robber Whiskey,750.0,44.99,32.14,210,6749.40,140.0,3194.29,6298.60,110.33,1990,41036.44,202.50,-450.80,-0.071571,0.933209,0.666667
10645,173357,TAMWORTH DISTILLING,3666,Art in the Age Chicory Root,375.0,24.99,18.79,520,9770.80,360.0,4873.05,8996.40,141.19,1990,41036.44,202.50,-774.40,-0.086079,0.920743,0.692308
10646,173357,TAMWORTH DISTILLING,3848,Chicory Root Vodka,750.0,30.99,23.30,28,652.40,6.0,92.97,185.94,4.71,1990,41036.44,202.50,-466.46,-2.508659,0.285009,0.214286


In [71]:
cursor=conn.cursor()

In [72]:
cursor.execute("""
CREATE TABLE sale_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    Volume DECIMAL(15,2),
    actual_price DECIMAL(15,2),
    PurchasePrice DECIMAL(15,2),
    total_quantity_purchase INT,
    total_dollar_purchase DECIMAL(15,2),
    total_Quantity_sale DECIMAL(15,2),
    total_sale_price DECIMAL(15,2),
    total_sale_dollar DECIMAL(15,2),
    total_excise_tax DECIMAL(15,2),
    total_quantity_invoice INT,
    total_dollar_invoice DECIMAL(15,2),
    total_freight_cost DECIMAL(15,2),
    gross_profit DECIMAL(15,2),
    margin DECIMAL(15,2),
    turnOver DECIMAL(15,2),
    sale_to_purchase_ratio DECIMAL(15,2)
);

""")

In [75]:
pd.read_sql(""" SELECT * FROM sale_summary """,conn)

,VendorNumber,VendorName,Brand,Description,Volume,actual_price,PurchasePrice,total_quantity_purchase,total_dollar_purchase,total_Quantity_sale,total_sale_price,total_sale_dollar,total_excise_tax,total_quantity_invoice,total_dollar_invoice,total_freight_cost,gross_profit,margin,sale_to_purchase_ratio,turnOver
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",90085,Ch Lilian 09 Ladouys St Este,750.0,36.99,23.86,8,190.88,18.0,295.92,665.82,2.00,328,5630.88,27.08,474.94,0.713316,3.488160,2.250000
1,2,"IRA GOLDMAN AND WILLIAMS, LLP",90609,Flavor Essence Variety 5 Pak,162.5,24.99,17.00,320,5440.00,24.0,449.82,599.76,0.52,328,5630.88,27.08,-4840.24,-8.070295,0.110250,0.075000
2,54,AAPER ALCOHOL & CHEMICAL CO,990,Ethyl Alcohol 200 Proof,3750.0,134.49,105.07,1,105.07,0.0,0.00,0.00,0.00,1,105.07,0.48,-105.07,0.000000,0.000000,0.000000
3,60,ADAMBA IMPORTS INTL INC,771,Bak's Krupnik Honey Liqueur,750.0,14.99,11.44,39,446.16,47.0,494.67,704.53,37.01,4732,76770.25,367.52,258.37,0.366727,1.579097,1.205128
4,60,ADAMBA IMPORTS INTL INC,3401,Vesica Vodka,1750.0,14.99,11.10,6,66.60,0.0,0.00,0.00,0.00,4732,76770.25,367.52,-66.60,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,172662,SWEETWATER FARM,4215,Kingfish Silver Rum,750.0,25.99,19.40,224,4345.60,129.0,2702.96,3352.71,101.75,1629,34708.03,178.34,-992.89,-0.296146,0.771518,0.575893
10644,173357,TAMWORTH DISTILLING,2804,Camp Robber Whiskey,750.0,44.99,32.14,210,6749.40,140.0,3194.29,6298.60,110.33,1990,41036.44,202.50,-450.80,-0.071571,0.933209,0.666667
10645,173357,TAMWORTH DISTILLING,3666,Art in the Age Chicory Root,375.0,24.99,18.79,520,9770.80,360.0,4873.05,8996.40,141.19,1990,41036.44,202.50,-774.40,-0.086079,0.920743,0.692308
10646,173357,TAMWORTH DISTILLING,3848,Chicory Root Vodka,750.0,30.99,23.30,28,652.40,6.0,92.97,185.94,4.71,1990,41036.44,202.50,-466.46,-2.508659,0.285009,0.214286


In [74]:
sale_summary.to_sql('sale_summary',conn,if_exists='replace',index=False)

10648

In [76]:
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time
import numpy as np
from ingestion_db import ingest_db

# ---------------- Logging Setup ---------------- #
os.makedirs("logs", exist_ok=True)
logging.basicConfig(
    filename="logs/sale_summary_script.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# ---------------- Create Sale Summary ---------------- #
def create_sale_summary(conn):
    """Merge multiple tables into a complete vendor sale summary."""
    query = """
    WITH 
    freight_summary AS (
        SELECT 
            VendorNumber,
            SUM(Quantity) AS total_quantity_invoice,
            SUM(Dollars) AS total_dollar_invoice,
            SUM(Freight) AS total_freight_cost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),
    purchase_summary AS (
        SELECT
            t1.VendorNumber,
            t1.VendorName,
            t1.Brand,
            t2.Description,
            t2.Volume,
            t2.Price AS actual_price,
            t2.PurchasePrice,    
            SUM(t1.Quantity) AS total_quantity_purchase,
            SUM(t1.Dollars) AS total_dollar_purchase
        FROM purchases t1
        JOIN purchase_price t2
            ON t1.VendorNumber = t2.VendorNumber 
            AND t1.Brand = t2.Brand
        WHERE t1.PurchasePrice > 0
        GROUP BY t1.VendorNumber, t1.Brand, t1.VendorName, 
                 t2.Volume, t2.Price, t2.PurchasePrice, t2.Description
    ),
    sale_summary AS (
        SELECT 
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS total_Quantity_sale,
            SUM(SalesDollars) AS total_sale_dollar,
            SUM(SalesPrice) AS total_sale_price,
            SUM(ExciseTax) AS total_excise_tax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.Volume,
        p.actual_price,
        p.PurchasePrice,
        p.total_quantity_purchase,
        p.total_dollar_purchase,
        s.total_Quantity_sale,
        s.total_sale_price,
        s.total_sale_dollar,
        s.total_excise_tax,
        f.total_quantity_invoice,
        f.total_dollar_invoice,
        f.total_freight_cost
    FROM purchase_summary p
    LEFT JOIN sale_summary s
        ON p.VendorNumber = s.VendorNo AND p.Brand = s.Brand
    LEFT JOIN freight_summary f
        ON p.VendorNumber = f.VendorNumber;
    """
    return pd.read_sql_query(query, conn)

# ---------------- Clean & Feature Engineering ---------------- #
def clean_data(df):
    """Clean and transform the sale summary DataFrame."""
    df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
    df.fillna(0, inplace=True)

    # --- Feature Engineering ---
    df['gross_profit'] = df['total_sale_dollar'] - df['total_dollar_purchase']
    df['margin'] = np.where(df['total_sale_dollar'] != 0,
                            df['gross_profit'] / df['total_sale_dollar'], 0)
    df['turnOver'] = np.where(df['total_quantity_purchase'] != 0,
                              df['total_Quantity_sale'] / df['total_quantity_purchase'], 0)
    df['sale_to_purchase_ratio'] = np.where(df['total_dollar_purchase'] != 0,
                                            df['total_sale_dollar'] / df['total_dollar_purchase'], 0)

    return df

# ---------------- Main Script ---------------- #
if __name__ == '__main__':
    try:
        start = time.time()
        conn = sqlite3.connect('inventory.db')
        logging.info("Database connection established.")

        logging.info("Creating Vendor Summary Table ...")
        summary_df = create_sale_summary(conn)
        logging.info(f"Summary DataFrame created with shape {summary_df.shape}")

        logging.info("Cleaning and transforming data ...")
        clean_df = clean_data(summary_df)
        logging.info(f"Cleaned DataFrame ready with shape {clean_df.shape}")

        logging.info("Ingesting data into database ...")
        ingest_db(clean_df, 'sale_summary', conn)
        logging.info("Data ingestion completed successfully.")

        end = time.time()
        logging.info(f"Total execution time: {round(end - start, 2)} seconds")

    except Exception as e:
        logging.error(f"Error occurred: {e}", exc_info=True)
    finally:
        conn.close()
        logging.info("Database connection closed.")


### statistics summary
- Negative and Zero values
  . Gross Profit- min value(-52002.78) indicate the loss. some product or transection may be selling at a loss due to high cost or selling at discount        price lower than the purchase price
  - Profit Margin : infinite and nan values. which suggested that revenue is 0 or lower than cost
  - Total Sale Quantity and sale dollars: minimmum value is 0,meaning some product is purchase but not sell at that time
### outlier indicate the High Standard deviation 
- purchase and actual price: the max value (5681.81 and 7499.99) are significantaly higher than the mean(24.39 & 35.64) indicating potential premium
- frieght cost : huge variation ,from 0.09 to 257,032.07. suggesting logistics inefficeincy or bulk shipment
- stock turnOver : range from 0 to 274.5 ,
- 
